In [103]:
import pandas as pd
import datetime
import ccxt

In [104]:
print(ccxt.exchanges)

['ace', 'alpaca', 'ascendex', 'bequant', 'bigone', 'binance', 'binancecoinm', 'binanceus', 'binanceusdm', 'bingx', 'bit2c', 'bitbank', 'bitbay', 'bitbns', 'bitcoincom', 'bitfinex', 'bitfinex2', 'bitflyer', 'bitforex', 'bitget', 'bithumb', 'bitmart', 'bitmex', 'bitopro', 'bitpanda', 'bitrue', 'bitso', 'bitstamp', 'bitstamp1', 'bittrex', 'bitvavo', 'bl3p', 'blockchaincom', 'btcalpha', 'btcbox', 'btcmarkets', 'btctradeua', 'btcturk', 'bybit', 'cex', 'coinbase', 'coinbaseprime', 'coinbasepro', 'coincheck', 'coinex', 'coinfalcon', 'coinmate', 'coinone', 'coinsph', 'coinspot', 'cryptocom', 'currencycom', 'delta', 'deribit', 'digifinex', 'exmo', 'fmfwio', 'gate', 'gateio', 'gemini', 'hitbtc', 'hitbtc3', 'hollaex', 'huobi', 'huobijp', 'huobipro', 'idex', 'independentreserve', 'indodax', 'kraken', 'krakenfutures', 'kucoin', 'kucoinfutures', 'kuna', 'latoken', 'lbank', 'lbank2', 'luno', 'lykke', 'mercado', 'mexc', 'mexc3', 'ndax', 'novadax', 'oceanex', 'okcoin', 'okex', 'okex5', 'okx', 'paymium'

In [105]:
exchange = ccxt.binance()

In [106]:
# Retrieve trading pairs
markets = exchange.load_markets()

# Create a DataFrame from the list of trading pairs
df_currencies = pd.DataFrame(list(markets.values()))

In [107]:
# If you want to modify the original DataFrame in-place to remove rows not equal to 'EUR', you can do:
df_currencies = df_currencies[df_currencies['quote'] == 'USD']
df_currencies = df_currencies[df_currencies['type'] == 'swap']

# If you want to reset the index after dropping rows, you can use the reset_index method:
df_currencies = df_currencies.reset_index(drop=True)
df_currencies.head()
df_currencies.to_excel('Test.xlsx')

In [108]:
df_currencies['id'].describe()

count              41
unique             41
top       BTCUSD_PERP
freq                1
Name: id, dtype: object

In [154]:
df_currencies.to_excel('Currencies.xlsx')

Retrieval of Data in a set period

Issue in the code.. period definition is not working properly

In [159]:
# Define a list of trading pairs
symbols = ['BTC/USD', 'ETH/USD', 'LTC/USD', 'XRP/USD']  # Replace with your desired list of trading pairs

# Specify the timeframe
timeframe = '5m'  # Replace with the desired timeframe

start_date = exchange.parse8601('2023-09-01T00:00:00Z')  # Start date in ISO8601 format
end_date = exchange.parse8601('2023-09-30T00:00:00Z')  # End date in ISO8601 format

limit = 1000  # Number of data points to fetch per request

# ... (previous code)

# Create an empty DataFrame to store the data
df = pd.DataFrame()


while start_date < end_date:
    data_available = False  # Flag to check if data is available for at least one symbol
    max_timestamp = None  # To keep track of the maximum timestamp
    
    for symbol in symbols:
        # Calculate the end timestamp for this data request
        end_timestamp = min(start_date + limit * 300 * 1000, end_date)
        
        # Fetch the OHLCV data for each trading pair within the specified date range
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, start_date, limit, params={'endTime': end_timestamp})
        
        if ohlcv:
            data_available = True  # Data is available for at least one symbol
            
            # Extract the 'Open' and 'Volume' values and store them in a DataFrame
            open_values = [candle[1] for candle in ohlcv]
            volume_values = [candle[5] for candle in ohlcv]
            
            # Create a DataFrame for the current pair
            df_pair = pd.DataFrame({
                'Timestamp': [candle[0] for candle in ohlcv],
                f'{symbol}_Open': open_values,  # Rename the open column
                f'{symbol}_Volume': volume_values  # Create a new volume column
            })
            
            if df.empty:
                df = df_pair
            else:
                # Merge the DataFrames on the 'Timestamp' column with an inner join
                df = df.merge(df_pair, on='Timestamp', how='inner')
            
            # Update the maximum timestamp for all pairs
            max_symbol_timestamp = ohlcv[-1][0]
            if max_timestamp is None or max_symbol_timestamp > max_timestamp:
                max_timestamp = max_symbol_timestamp
        
    # Update the start time for the next request
    start_date = end_timestamp + 1  # Increment by the entire limit duration
    
    # Check if there are more data points to fetch, if not, exit the loop
    if not data_available or start_date >= end_date:
        break

# Sort the DataFrame by 'Timestamp'
df.sort_values(by='Timestamp', inplace=True)

# Convert the 'Timestamp' column to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')

# Reset the index
df.reset_index(drop=True, inplace=True)

# Export the DataFrame to an Excel file
df.to_excel('Pair Data.xlsx')

df

,Timestamp,ETH/USD_Open,ETH/USD_Volume,LTC/USD_Open,LTC/USD_Volume,XRP/USD_Open,XRP/USD_Volume
0,2023-09-28 18:45:00,1653.52,5949.603474,64.65,4808.490321,0.5056,952338.837455
1,2023-09-28 18:50:00,1647.75,1848.190226,64.48,2481.295795,0.5051,204559.405982
2,2023-09-28 18:55:00,1649.87,2408.636010,64.55,2228.581354,0.5054,261033.944107
3,2023-09-28 19:00:00,1649.69,1664.695934,64.59,3486.557318,0.5058,327899.713751
4,2023-09-28 19:05:00,1648.89,868.291689,64.57,1595.892812,0.5057,77448.268102
...,...,...,...,...,...,...,...
347,2023-09-29 23:40:00,1667.13,15.763962,65.58,43.769994,0.5198,230251.138505
348,2023-09-29 23:45:00,1666.99,117.602956,65.57,1125.371810,0.5201,136066.880681
349,2023-09-29 23:50:00,1667.02,32.385691,65.61,193.908625,0.5206,321026.619023
350,2023-09-29 23:55:00,1666.80,124.853488,65.58,1190.775108,0.5213,200663.427078


The choice of a time frame for a momentum trading bot depends on your trading strategy, risk tolerance, and the assets you are trading. Different time frames offer different trade-offs, and there is no one-size-fits-all answer. Here are some considerations for selecting a time frame:

1-Minute (or Less) Time Frame:
- Very short time frames like 1 minute are often used by high-frequency traders (HFT) and algorithmic trading systems that execute many trades in a short time. They require low latency and advanced infrastructure.
- High-frequency trading is associated with increased transaction costs, such as trading fees and potential slippage, and may not be suitable for all traders.
- The price action can be more erratic, making it challenging to identify meaningful trends, and there's a risk of "noise" in the data.
  
Shorter Time Frames (5-Minute, 15-Minute, 30-Minute):
- These time frames are often used by day traders and short-term momentum traders.
- They allow for more trades throughout the day but still require careful risk management due to the rapid price changes.

Longer Time Frames (1-Hour, 4-Hour, Daily):
- Longer time frames are generally used by swing traders and medium-term traders.
- Trends and patterns on these time frames tend to be more reliable, with less "noise" compared to shorter time frames.
- Trades occur less frequently, and holding positions for longer periods may require more patience.
- When choosing a time frame for a momentum trading bot, consider the following factors:

Trading Style: Your trading style and objectives matter. If you want to capitalize on very short-term price movements, you might choose a shorter time frame. For longer-term trades, consider longer time frames.

Asset Volatility: The volatility of the asset you're trading is crucial. Highly volatile assets might require shorter time frames to capture momentum, while less volatile assets can be traded on longer time frames.

Risk Management: Shorter time frames often require tighter stop-loss orders and more frequent monitoring, so you need a robust risk management strategy.

Technical Analysis: The time frame you choose should align with your technical analysis. Different time frames may show different patterns and signals.

Infrastructure: High-frequency trading often demands low-latency infrastructure, including high-speed internet connections and powerful servers.

Backtesting: Use backtesting to evaluate the performance of your trading strategy on historical data across various time frames. This can help you determine which time frame works best for your specific strategy.